Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
pip install matplotlib pandas numpy scikit-learn azureml-sdk azureml-dataprep numba

Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-269475
aml-quickstarts-269475
southcentralus
81cefad3-d2c9-4f77-a466-99a7f541c7bb


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [4]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'azmlproject2'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
azmlproject2,quick-starts-ws-269475,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "azmlproject22"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [6]:
#pip install --upgrade azureml-sdk azureml-dataprep


In [7]:
pip install azureml-dataprep

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bankmarketing"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [9]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [10]:
automl_settings = {
    "experiment_timeout_hours": 0.5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "iteration_timeout_minutes": 4
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [11]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [12]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [13]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [14]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [b3532b43][1b91cb39-e536-4bd8-acc7-39556560415d], (This step will run and generate new outputs)
Submitted PipelineRun 8d7fa654-b9a3-493c-9a3a-cc74b9823796
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8d7fa654-b9a3-493c-9a3a-cc74b9823796?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-269475/workspaces/quick-starts-ws-269475&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [15]:
#pip install --upgrade numba


In [16]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

2024-11-04 15:21:07.874076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 15:21:08.935258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 15:21:09.254830: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 15:21:11.866486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-04 15:21:18.782331: W tensorflow/comp

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [17]:
pipeline_run.wait_for_completion()

PipelineRunId: 8d7fa654-b9a3-493c-9a3a-cc74b9823796
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/8d7fa654-b9a3-493c-9a3a-cc74b9823796?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-269475/workspaces/quick-starts-ws-269475&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: d7914394-0d83-4e45-8dd6-79516da1ee4b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d7914394-0d83-4e45-8dd6-79516da1ee4b?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-269475/workspaces/quick-starts-ws-269475&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [18]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/d7914394-0d83-4e45-8dd6-79516da1ee4b/metrics_data, 1 files out of an estimated total of 1


In [19]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,d7914394-0d83-4e45-8dd6-79516da1ee4b_2,d7914394-0d83-4e45-8dd6-79516da1ee4b_20,d7914394-0d83-4e45-8dd6-79516da1ee4b_0,d7914394-0d83-4e45-8dd6-79516da1ee4b_4,d7914394-0d83-4e45-8dd6-79516da1ee4b_23,d7914394-0d83-4e45-8dd6-79516da1ee4b_3,d7914394-0d83-4e45-8dd6-79516da1ee4b_1,d7914394-0d83-4e45-8dd6-79516da1ee4b_25,d7914394-0d83-4e45-8dd6-79516da1ee4b_27,d7914394-0d83-4e45-8dd6-79516da1ee4b_29,...,d7914394-0d83-4e45-8dd6-79516da1ee4b_17,d7914394-0d83-4e45-8dd6-79516da1ee4b_14,d7914394-0d83-4e45-8dd6-79516da1ee4b_19,d7914394-0d83-4e45-8dd6-79516da1ee4b_16,d7914394-0d83-4e45-8dd6-79516da1ee4b_9,d7914394-0d83-4e45-8dd6-79516da1ee4b_18,d7914394-0d83-4e45-8dd6-79516da1ee4b_38,d7914394-0d83-4e45-8dd6-79516da1ee4b_36,d7914394-0d83-4e45-8dd6-79516da1ee4b_21,d7914394-0d83-4e45-8dd6-79516da1ee4b_15
accuracy,[0.9001517450682853],[0.9019726858877086],[0.9101669195751139],[0.9071320182094081],[0.7292867981790592],[0.8883156297420334],[0.9116843702579667],[0.9101669195751139],[0.8995447647951441],[0.9068285280728376],...,[0.8024279210925644],[0.9113808801213961],[0.8880121396054628],[0.8437025796661608],[0.7371775417298938],[0.9132018209408195],[0.9080424886191198],[0.9001517450682853],[0.9104704097116844],[0.9068285280728376]
weighted_accuracy,[0.9763404704059276],[0.9607196759303707],[0.9512815952194833],[0.9720244705544736],[0.7037412575392604],[0.9843874836349229],[0.952375685487909],[0.9539274862816189],[0.9577343823385595],[0.9702181178635071],...,[0.7909905874434631],[0.9646807517949991],[0.9843450583187134],[0.8411526027126678],[0.7383589358810905],[0.9546457273395061],[0.9477507511522958],[0.9581132208667712],[0.9557338389725855],[0.9805076942162568]
precision_score_weighted,[0.8827113977984437],[0.8875607647592548],[0.9051980543721705],[0.8930395334907932],[0.9146897003922134],[0.9007928150046113],[0.9067326786550023],[0.903605295208037],[0.8851714813984484],[0.892553572081738],...,[0.9145334128008851],[0.9004482554668507],[0.788565560086672],[0.9162625570891886],[0.8774156959060226],[0.907597716175493],[0.904387168366052],[0.8859481117753807],[0.9030335205195396],[0.8973958505740813]
norm_macro_recall,[0.1865537828310615],[0.3307001798657767],[0.48912840119515355],[0.2915131509483242],[0.6643595314968871],[0.0027100271002709064],[0.4955737458946703],[0.4678140287896384],[0.3303343354691237],[0.30301270545172976],...,[0.6969909993016541],[0.393396647568663],[0.0],[0.7079468812459826],[0.46483818563407775],[0.49254603619173576],[0.49620911109999666],[0.33338612606905293],[0.45631447428623284],[0.22012347943028288]
balanced_accuracy,[0.5932768914155307],[0.6653500899328884],[0.7445642005975768],[0.6457565754741621],[0.8321797657484435],[0.5013550135501355],[0.7477868729473351],[0.7339070143948192],[0.6651671677345619],[0.6515063527258649],...,[0.848495499650827],[0.6966983237843315],[0.5],[0.8539734406229913],[0.7324190928170389],[0.7462730180958679],[0.7481045555499983],[0.6666930630345265],[0.7281572371431164],[0.6100617397151414]
recall_score_macro,[0.5932768914155307],[0.6653500899328884],[0.7445642005975768],[0.6457565754741621],[0.8321797657484435],[0.5013550135501355],[0.7477868729473351],[0.7339070143948192],[0.6651671677345619],[0.6515063527258649],...,[0.848495499650827],[0.6966983237843315],[0.5],[0.8539734406229913],[0.7324190928170389],[0.7462730180958679],[0.7481045555499983],[0.6666930630345265],[0.7281572371431164],[0.6100617397151414]
matthews_correlation,[0.33340661446628406],[0.41778310629483134],[0.5216286298277554],[0.42149687086703563],[0.4327991759024445],[0.049063912865329386],[0.52924365161229],[0.5120524394053735],[0.4088131940866044],[0.42495912753824355],...,[0.4849231250651206],[0.4839499450987893],[0.0],[0.5238288494614208],[0.3159771519617247],[0.5328108142609521],[0.5184688170203535],[0.4125900108715406],[0.5079177049128708],[0.39340196123415067]
recall_score_micro,[0.9001517450682853],[0.9019726858877086],[0.9101669195751139],[0.9071320182094081],[0.7292867981790592],[0.8883156297420334],[0.9116843702

### Retrieve the Best Model

In [20]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/d7914394-0d83-4e45-8dd6-79516da1ee4b/model_data, 1 files out of an estimated total of 1


In [21]:
pip install azureml-sdk[automl,notebooks,explain,train]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.8/934.8 kB 19.5 MB/s eta 0:00:0000:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.2.0
    Uninstalling importlib_metadata-8.2.0:
      Successfully uninstalled importlib_metadata-8.2.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2024.9.0
    Uninstalling distributed-2024.9.0:
      Successfully uninstalled distributed-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
dask-sql 2024.5.0 requ

In [22]:
pip install scipy==1.10.1

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 16.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
azureml-training-tabular 1.57.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.14.1 which is incompatible.
azureml-train-automl-runtime 1.57.0 requires scipy<=1.11.0,>=1.0.0, but you have scipy 1.14.1 which is incompatible.
azureml-automl-runtime 1.57.0 requires scipy<=1.11.0,>=1.0.0, but you have scipy 1.14.1 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
Note: you may need to restart the kernel to use 

In [28]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install scipy==1.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 19.2 MB/s eta 0:00:0000:0100:01
Reason for being yanked: License Violation
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
azureml-training-tabular 1.57.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [35]:
import pickle
import scipy  # Assuming a compatible version is installed

with open(best_model_output._path_on_datastore, "rb") as f:
    best_model = pickle.load(f)

print(best_model)  # Or perform other operations with the loaded model

Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/azmlproject222/code/Users/odl_user_269475')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('32', Pipeline(steps=[('standardscale...fo=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))]))], flatten_transform=False, weights=[0.16666666666666666, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333]))])
Y_transformer(['LabelEncoder', LabelEncoder()])


In [36]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('32', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.7708333333333335, reg_lambda=1.5625, subsample=0.5, tree_method='auto'))], verbose=False)), ('33', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=Tru

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [45]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [46]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [47]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28794,464
1,1406,2286


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [48]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,4eb12566-2e66-4cff-92dd-7de575978b6c,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [49]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [50]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [51]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  e7bd4f82-1767-4462-b647-6a956b6ee6d0


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [52]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
